In [ ]:
"""
Created on Thu Apr 21 14:17 2022

Combine all info per ice shelf and make small panda dataframes

Author: @claraburgard
"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import basal_melt_neural_networks.data_formatting as dfmt

In [ ]:
nemo_run = 'OPM006'

In [ ]:
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath_nn = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'
inputpath_tides = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/TIDES/'

In [ ]:
# dIF, dGL
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
map_lim = [-3000000,3000000]
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = dfmt.cut_domain_stereo(file_mask_orig, map_lim, map_lim)
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = dfmt.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = dfmt.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
# bathymetry, ice draft, concentration
file_bed_orig = file_mask_orig_cut['bathy_metry']
file_bed_corr = file_other_cut['corrected_isf_bathy']
file_draft = file_other_cut['corrected_isfdraft'] 
file_bed_goodGL = file_bed_orig.where(file_draft < file_bed_orig,file_bed_corr)
file_isf_conc = file_conc_cut['isfdraft_conc']

In [ ]:
file_slope = xr.open_dataset(inputpath_mask+'nemo_5km_slope_info_bedrock_draft_latlon.nc')

In [ ]:
utide_file = xr.open_dataset(inputpath_tides + 'tidal_velocity_nemo_Ant_stereo.nc').rename({'ttv':'u_tide'})
u_tide = dfmt.cut_domain_stereo(utide_file['u_tide'], map_lim, map_lim)

In [ ]:
if nemo_run == 'OPM031':
    NEMO_melt_rates_2D = xr.open_mfdataset(outputpath_melt+'melt_rates_2D_NEMO.nc').isel(time=range(30,70))
else:
    NEMO_melt_rates_2D = xr.open_mfdataset(outputpath_melt+'melt_rates_2D_NEMO.nc')

melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']

In [ ]:
geometry_2D = file_isf[['dGL', 'dIF']].merge(file_draft).merge(file_bed_goodGL).merge(file_slope).merge(file_isf_conc).merge(u_tide) 
geometry_2D['dIF'] = geometry_2D['dIF'].where(np.isfinite(geometry_2D['dIF']), np.nan)
time_dpdt_in = melt_rate

In [ ]:
final_input_xr_isf

In [ ]:
merged_df = None
for kisf in tqdm(file_isf.Nisf.sel(Nisf=[23,66])):
    geometry_2D_isf = geometry_2D.where(file_isf['ISF_mask']==kisf,drop=True).drop('longitude').drop('latitude')
    melt_rate_isf = melt_rate.where(file_isf['ISF_mask']==kisf,drop=True)
    T_S_2D_isf = xr.open_dataset(inputpath_profiles+'flattened_T_S_profiles_isf'+str(kisf.values).zfill(3)+'.nc').squeeze().transpose('time','y','x').drop('longitude').drop('latitude')
    
    time_dpdt_in = T_S_2D_isf.merge(melt_rate_isf).drop('longitude').drop('latitude')
    geometry_2D_br_isf, time_dpdt_in_br = xr.broadcast(geometry_2D_isf,time_dpdt_in)
    final_input_xr_isf = xr.merge([geometry_2D_br_isf, time_dpdt_in_br]).transpose('y','x','time').drop('Nisf')
    ds_kisf = final_input_xr_isf

    df_kisf = ds_kisf.to_dataframe()
    clean_df_kisf = df_kisf.dropna()
    clean_df_kisf = clean_df_kisf.where(clean_df_kisf['S_003']!=0).dropna()

    if merged_df is None:
        merged_df = clean_df_kisf.copy()
    else:
        merged_df = merged_df.append(clean_df_kisf)

In [ ]:
merged_df

In [ ]:
    geometry_2D_br, time_dpdt_in_br = xr.broadcast(geometry_2D,time_dpdt_in)
    ds_kisf = final_input_xr.where(file_isf['ISF_mask']==kisf,drop=True).drop('longitude').drop('latitude').drop('Nisf')
    df_kisf = ds_kisf.to_dataframe()
    clean_df_kisf = df_kisf.dropna()
    clean_df_kisf = clean_df_kisf.where(clean_df_kisf['S_003']!=0).dropna()
    if df_all is None:
        df_all = clean_df_kisf.copy()
    else:
        df_all = df_all.append(clean_df_kisf)
    
merged_df = df_all